In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
tf.reset_default_graph()

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ANANT/apal/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#hyper parameters


encoder_vocab_size = 69
decoder_vocab_size = 41


encoder_embedding_size = 50
decoder_embedding_size = 50

encoder_hidden_size = 150
decoder_hidden_size = 150



In [3]:
encoder_input = tf.placeholder(name='encoder_in',dtype=tf.int32,shape=[None,None])
decoder_input = tf.placeholder(name='decoder_in',dtype=tf.int32,shape=[None,None])

decoder_target= tf.placeholder(name='decoder_target',dtype=tf.int32,shape=[None,None])


encoder_embedding = tf.get_variable(name='encoder_embedd',
                                    shape=[encoder_vocab_size,encoder_embedding_size],
                                    dtype=tf.float32,
                                    initializer=tf.random_uniform_initializer(-0.01,0.01))

decoder_embedding = tf.get_variable(name='decoder_embedd',shape=[decoder_vocab_size,decoder_embedding_size],
                                    dtype=tf.float32,
                                    initializer=tf.random_uniform_initializer(-0.01,0.01))

embedd_encoder= tf.nn.embedding_lookup(encoder_embedding,encoder_input)
embedd_decoder= tf.nn.embedding_lookup(decoder_embedding,decoder_input)



In [4]:
encoder_cell = rnn.LSTMCell(num_units=encoder_hidden_size)
encoder_cel =  rnn.DropoutWrapper(encoder_cell,output_keep_prob=0.5)
decoder_cell = rnn.LSTMCell(num_units= decoder_hidden_size)
decoder_cel = rnn.DropoutWrapper(decoder_cell,output_keep_prob=0.5)

In [5]:
encoder_network,encoder_last_state = tf.nn.dynamic_rnn(cell=encoder_cel,
                                    inputs=embedd_encoder,
                                    dtype=tf.float32,
                                    time_major=True)

decoder_network,decoder_last_state = tf.nn.dynamic_rnn(cell=decoder_cel,
                                    inputs=embedd_decoder,
                                    dtype=tf.float32,
                                    time_major=True,
                                    initial_state=encoder_last_state,scope='decoder_out')

linear_projection = tf.contrib.layers.fully_connected(decoder_network,decoder_vocab_size)

prediction = tf.argmax(linear_projection,2)

cross_entropy= tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(decoder_target,
                                                                         depth=decoder_vocab_size,
                                                                         dtype=tf.float32),
                                                       logits=linear_projection)

loss= tf.reduce_mean(cross_entropy)

train= tf.train.AdamOptimizer().minimize(loss)


# encoder_input_ = np.random.randint(0,10,[5,3])  #time major  [max_time , batch]
# print("encoder_input \n",encoder_input_)

# decoder_input_ = np.random.randint(0,10,[4,3])  #time_major  [max_time , batch]
# print("decoder_input \n",decoder_input_)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     prediction_data = sess.run(prediction,feed_dict={encoder_input:encoder_input_,decoder_input:decoder_input_})
#     print("prediction_data \n",prediction_data)


In [6]:

import numpy as np

index_to_ph = {0: 'PAD', 1: 'AA0', 2: 'AA1', 3: 'AA2', 4: 'AE0', 5: 'AE1', 6: 'AE2', 7: 'AH0', 
               8: 'AH1', 9: 'AH2', 10: 'AO0', 11: 'AO1', 12: 'AO2', 13: 'AW0', 14: 'AW1', 
               15: 'AW2', 16: 'AY0', 17: 'AY1', 18: 'AY2', 19: 'B', 20: 'CH', 21: 'D', 
               22: 'DH', 23: 'EH0', 24: 'EH1', 25: 'EH2', 26: 'ER0', 27: 'ER1', 28: 'ER2', 
               29: 'EY0', 30: 'EY1', 31: 'EY2', 32: 'F', 33: 'G', 34: 'HH', 35: 'IH0', 
               36: 'IH1', 37: 'IH2', 38: 'IY0', 39: 'IY1', 40: 'IY2', 41: 'JH', 42: 'K', 
               43: 'L', 44: 'M', 45: 'N', 46: 'NG', 47: 'OW0', 48: 'OW1', 49: 'OW2', 
               50: 'OY0', 51: 'OY1', 52: 'OY2', 53: 'P', 54: 'R', 55: 'S', 56: 'SH', 
               57: 'T', 58: 'TH', 59: 'UH0', 60: 'UH1', 61: 'UH2', 62: 'UW0', 63: 'UW1', 
               64: 'UW2', 65: 'V', 66: 'W', 67: 'Y', 68: 'Z', 69: 'ZH'}


ph_to_index = {j:i for i,j in index_to_ph.items()}



index_to_words= {0: 'PAD', 1: 'GO', 2: 'EOS', 3: '+', 4: '0', 5: '1', 6: '2', 7: '3', 
                 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: 'a', 15: 'b', 
                 16: 'c', 17: 'd', 18: 'e', 19: 'f', 20: 'g', 21: 'h', 22: 'i', 23: 'j', 
                 24: 'k', 25: 'l', 26: 'm', 27: 'n', 28: 'o', 29: 'p', 30: 'q', 31: 'r', 
                 32: 's', 33: 't', 34: 'u', 35: 'v', 36: 'w', 37: 'x', 38: 'y', 39: 'z', 
                 40: 'à', 41: 'é'}


words_to_index = {j:i for i,j in index_to_words.items()}


In [27]:
# import pickle as pk
# import random
# import numpy as np

# with open('encoded_data_pad','rb') as f:
#     file_opn=pk.load(f)

# random.shuffle(file_opn)

In [28]:
# train_data_int=int(len(file_opn)*0.90)

# train_data = file_opn[:train_data_int][:30000]
# test_data = file_opn[train_data_int:][:5000]

# np.save('train_data_p',train_data)
# np.save('test_data_p',test_data)

In [7]:
import numpy as np
import random
train_da= np.load('train_data_p.npy')
test_da = np.load('test_data_p.npy')



def get_train_batch(i,batch_size=None):
    batch_d= train_da[i*batch_size:(i+1)*batch_size]
    phe_n=[]
    words=[]
    for i in batch_d:
        words.append(i[0])
        phe_n.append(i[1])
#     print(phe_n)

    max_phe=max([len(i) for i in phe_n])
    max_words= max([len(i) for i in words])
    padded_phe =[i + [0]*(max_phe-len(i))  if len(i)<max_phe else i for i in phe_n]
    
    
    padded_words = [[words_to_index['GO']] + i + [0]*(max_words-len(i)) if len(i)< max_words else [words_to_index['GO']] + i for i in words]
    
  
    
    padded_target =[i + [words_to_index['EOS']] for i in words]
    max_target=max([len(i) for i in padded_target])
    padded_target_ =[i + [0]*(max_target-len(i))  if len(i)<max_target else i for i in padded_target]
 
    
    
    

    return {'encoder_in': np.transpose(np.array(padded_phe),[1,0]),
            'decoder_in':np.transpose(np.array(padded_words),[1,0]),
            'decoder_targ':np.transpose(np.array(padded_target_),[1,0])}
    
# def get_test_data():
#     nb=random.randint(1,len(test_da)-5)
#     print(nb)
#     inference=np.random.randint(0,0,[5])
#     print(inference)
#     return test_da[nb:nb+5]



# print(get_train_batch(1,batch_size=2))
# d=get_train_batch(1)
# print(d['encoder_in'].shape)
# print(d['decoder_in'].shape)
# print(d['decoder_targ'].shape)

#word,phe
#take input  == > pad == > convert it to time major 
#take output ===> pad ===> convert it to time major



In [8]:
from tqdm import tqdm_notebook as tq
batch_size = 100
epoch = 50
iteration = int(len(train_da)//batch_size)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epoch):
        for j in tq(range(iteration)):
            batch_d = get_train_batch(j,batch_size=35)
            feed_dict = {encoder_input:batch_d['encoder_in'],
                         decoder_input:batch_d['decoder_in'],
                         decoder_target:batch_d['decoder_targ']}
            trai,los = sess.run([train,loss],feed_dict)
        batch_dr = get_train_batch(j,batch_size=1)
        print({'epoch':i,'loss':los,
                   'predf':sess.run(prediction,feed_dict = {encoder_input:batch_dr['encoder_in'],decoder_input:batch_dr['decoder_in']}),
                   'actual': batch_dr['decoder_in']})


{'epoch': 0, 'loss': 1.8948039, 'predf': array([[22],
       [14],
       [22],
       [27],
       [18],
       [18],
       [18],
       [ 2],
       [ 2],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 1, 'loss': 1.7246128, 'predf': array([[32],
       [14],
       [31],
       [22],
       [18],
       [14],
       [18],
       [27],
       [ 2],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 2, 'loss': 1.6721714, 'predf': array([[31],
       [18],
       [27],
       [14],
       [18],
       [22],
       [18],
       [31],
       [32],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 3, 'loss': 1.5955616, 'predf': array([[17],
       [28],
       [31],
       [22],
       [18],
       [18],
       [18],
       [32],
       [ 2],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 4, 'loss': 1.5032647, 'predf': array([[16],
       [28],
       [31],
       [21],
       [18],
       [22],
       [18],
       [31],
       [32],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}


...


{'epoch': 45, 'loss': 0.31222683, 'predf': array([[23],
       [14],
       [16],
       [24],
       [32],
       [18],
       [22],
       [38],
       [ 2],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 46, 'loss': 0.34985253, 'predf': array([[23],
       [14],
       [16],
       [24],
       [32],
       [18],
       [18],
       [18],
       [33],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 47, 'loss': 0.32966763, 'predf': array([[23],
       [28],
       [16],
       [24],
       [32],
       [18],
       [18],
       [14],
       [ 2],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 48, 'loss': 0.31126088, 'predf': array([[20],
       [14],
       [16],
       [32],
       [32],
       [18],
       [18],
       [38],
       [ 2],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}



{'epoch': 49, 'loss': 0.32258323, 'predf': array([[23],
       [14],
       [16],
       [24],
       [18],
       [17],
       [18],
       [18],
       [33],
       [ 2]]), 'actual': array([[ 1],
       [23],
       [14],
       [16],
       [24],
       [32],
       [17],
       [18],
       [22],
       [33]])}
